# Imports and preliminary dataframe processing

#### Currently works when I use my base environment (anaconda3/bin/python)

In [1]:

import numpy as np
import torch
from torch import nn
import tqdm
import matplotlib.pyplot as plt
#from ax.service.managed_loop import optimize
from sklearn import model_selection
import json
import pandas as pd
from scipy.spatial import Delaunay
import random
from matplotlib import patches
import matplotlib.colors as colors
from sklearn.model_selection import GroupShuffleSplit
from shapely.geometry import Point
from alphashape import alphashape

df = pd.read_csv('no_avg_dataset.csv')


In [2]:

seqs = np.unique(df["Sequence"])
ps = np.unique(df["p"])

nn_input_array = np.empty((0,3))
nn_output_array = np.empty((0,2))

seq_array = np.empty((0,1))

seqs_list = []


for seq in seqs:
    seq_dataset = df[df["Sequence"] == seq]

    #Now to get the p=0 point
    p0_seq_dataset = seq_dataset[seq_dataset["p"] == 0]
    p0_z0 = p0_seq_dataset["Z0"]#.mean()
    p0_z1 = p0_seq_dataset["Z1"]#.mean()

    #I have the p=0 point, now I need to loop through the p values for each coordinate at that p

    for p in ps:
        p_seq_dataset = seq_dataset[seq_dataset["p"] == p]
        p_z0 = p_seq_dataset["Z0"]#.mean()
        p_z1 = p_seq_dataset["Z1"]#.mean()

        p_arr = np.full((5,1), p)
        #Now I can put everything into the arrays
        input_row = np.concatenate((np.asarray(p0_z0).reshape(-1,1), np.asarray(p0_z1).reshape(-1,1), p_arr), axis = 1)
        output_row = np.concatenate((np.asarray(p_z0).reshape(-1,1), np.asarray(p_z1).reshape(-1,1)), axis = 1)

        seq_row = np.asarray(p_seq_dataset["Sequence"]).reshape(-1,1)

        if output_row.shape != (5,2):
            # This is because there are around 5 cases where there are only 4 replicas, and I just put this bit in so it would still run while I figured
            # out how to edit this block to make it work with varying replica numbers
            break
    
        else:
            nn_input_array = np.vstack((nn_input_array, input_row))
            nn_output_array = np.vstack((nn_output_array, output_row))

            seq_array = np.vstack((seq_array, seq_row))

#Arrays with the input and output values for the MLP
nia_no_avg = nn_input_array # Net input array
noa_no_avg = nn_output_array # Net output array 

In [3]:
#Grouped splitting, grouped by sequence

nia = nia_no_avg
noa = noa_no_avg


test_val_splitter = GroupShuffleSplit(n_splits = 1, test_size = 0.6, random_state = 0)

test_val_indices, train_indices = next(test_val_splitter.split(nia, noa, groups = seq_array))

x_test_val, x_train = nia[test_val_indices], nia[train_indices]
y_test_val, y_train = noa[test_val_indices], noa[train_indices]
groups_test_val = seq_array[test_val_indices]

val_test_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
val_indices, test_indices = next(val_test_splitter.split(x_test_val, y_test_val, groups=groups_test_val))

x_val, x_test = x_test_val[val_indices], x_test_val[test_indices]
y_val, y_test = y_test_val[val_indices], y_test_val[test_indices]

test_seqs = seq_array[test_indices]
val_seqs = seq_array[val_indices]
#Turning them into tensors
x_train = torch.Tensor(x_train)
x_val = torch.Tensor(x_val)
y_train = torch.Tensor(y_train)
y_val = torch.Tensor(y_val)
x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

# Functions

In [ ]:
def get_rmse_by_p(p, y_pred, x_vals, y_vals):
    x_train_subset = []
    y_train_subset = []
    y_pred_subset = []
    for i, point in enumerate(x_vals):
        if round(float(point[2]),1) == p:
            x_train_subset.append(point)
            y_train_subset.append(y_vals[i])
            y_pred_subset.append(y_pred[i])
    
    y_pred_subset = torch.Tensor(torch.stack(y_pred_subset))
    y_train_subset = torch.Tensor(torch.stack(y_train_subset))
    rmse = torch.sqrt( torch.mean( (y_pred_subset - y_train_subset)**2 ) ).item()

    return p, rmse

### MLP code

In [ ]:
#Always run

class MLPRegressor(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=(100, ), activation=nn.ReLU()):
        torch.manual_seed(0)  # control random effects

        super(MLPRegressor, self).__init__()

        layers = []
        prev_layer_size = input_size
        for layer_size in hidden_size:
            layers.append(nn.Linear(prev_layer_size, layer_size))
            layers.append(activation)
            prev_layer_size = layer_size

        layers.append(nn.Linear(prev_layer_size, output_size))
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, x):
        x = self.fc_layers(x)
        return x
    
import tqdm

def train_model(model, data, optimizer, n_epochs):
    torch.manual_seed(0)

    criterion = nn.MSELoss()
    x_train, x_val, y_train, y_val = data

    # do the training
    pbar = tqdm.tqdm(np.arange(n_epochs))
    for epoch in pbar:
        # Reset gradients
        optimizer.zero_grad()
        # Forward pass
        yp = model(x_train)
        # Compute Loss
        loss = criterion(yp, y_train)
        # Backward pass
        loss.backward()
        optimizer.step()
        # Update progress bar
        pbar.set_postfix_str(f'loss: {loss.item():.3e}')

    return model

# create a data structure to convert from str input to nn function
activation_dict = {'tanh': nn.Tanh(),
                   'relu': nn.ReLU(),
                   'leaky_relu': nn.LeakyReLU(),
                   'sigmoid': nn.Sigmoid(),
                   'elu': nn.ELU()}

def generate_model_and_optimizer(params):
    # create a list of layer sizes from the start, end, and depth
    raw_dims = np.linspace(params["layer_i"], params["layer_f"], params["num_layers"])
    hidden_size = tuple(raw_dims.round().astype(int))
    # choose the activation function
    activation = activation_dict[params['activation']]

    # initialize the model
    model = MLPRegressor(3, 2, hidden_size, activation)

    # initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])

    # return both objects to calling function
    return model, optimizer

def mlp_fitness(params):
    # the evaluation function cannot accept arguments -- we hard-code them here

    data = (x_train, x_val, y_train, y_val)
    try:
        # train on training set
        model, optimizer = generate_model_and_optimizer(params)
        train_model(model, data, optimizer, params['n_epochs'])
        # evaluate on held-out validation set
        y_pred = model(x_val).detach()
        rmse = torch.sqrt( torch.mean( (y_pred - y_val)**2 ) ).item()
    except:
        rmse = 1e12
    rmse = min(rmse, 1e2)  # need to choose a threshold RMSE to handle errors
    return np.log(rmse)    # log will be better behaved for very small numbers





# Models

## Absolute position model

### Training model

In [ ]:
# Training model
with open('best_parameters_absolute.json', 'r') as f:
    best_parameters = json.load(f)

model, optimizer = generate_model_and_optimizer(best_parameters)
x_train = torch.Tensor(x_train)
x_val = torch.Tensor(x_val)
y_train = torch.Tensor(y_train)
y_val = torch.Tensor(y_val)
x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

data = (x_train, x_val, y_train, y_val)
train_model(model, data, optimizer, best_parameters['n_epochs'])
# train rmse
y_pred = model(x_train).detach()
rmse_train = torch.sqrt( torch.mean( (y_pred - y_train)**2 ) ).item()
# validation rmse
y_pred = model(x_val).detach()
rmse_val = torch.sqrt( torch.mean( (y_pred - y_val)**2 ) ).item()
# test rmse
y_pred = model(x_test).detach()
rmse_test = torch.sqrt( torch.mean( (y_pred - y_test)**2 ) ).item()
# report
print(f'\ntrain: {rmse_train:.3e} / val: {rmse_val:.3e} / test: {rmse_test:.3e}')

In [ ]:
#Saving model and best parameters
torch.save(model.state_dict(), 'embedding_model_noavg_absolute.pth')
with open('best_parameters_absolute.json', 'w') as f:
    json.dump(best_parameters, f)

### Getting RMSE as f(p)

In [ ]:
x_train = torch.Tensor(x_train)
x_val = torch.Tensor(x_val)
y_train = torch.Tensor(y_train)
y_val = torch.Tensor(y_val)
x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

#Train
y_pred_train = model(x_train).detach()

#Test
y_pred_test = model(x_test).detach()

#Val
y_pred_val = model(x_val).detach()

train_rmse_list = []
test_rmse_list = []
val_rmse_list = []

for p in ps:
    _, rmse_train = get_rmse_by_p(p, y_pred_train, x_train, y_train)
    _, rmse_test = get_rmse_by_p(p, y_pred_test, x_test, y_test)
    _, rmse_val = get_rmse_by_p(p, y_pred_val, x_val, y_val)

    train_rmse_list += [rmse_train]
    test_rmse_list += [rmse_test]
    val_rmse_list +=[rmse_val]

    print(f'\n p: {p}  train: {rmse_train:.3e} / val: {rmse_val:.3e} / test: {rmse_test:.3e}')

plt.plot(ps, train_rmse_list, label = "Train")
plt.plot(ps, test_rmse_list, label = "Test")
plt.plot(ps, val_rmse_list, label = "Validation")
plt.legend()
plt.title("Absolute Position Model RMSE")
plt.show()

### To load the saved model

In [ ]:
with open('best_parameters_absolute.json', 'r') as f:
    best_parameters = json.load(f)

#Initializing the model (maybe not the right wording here)
model, optimizer = generate_model_and_optimizer(best_parameters)
model_dict = torch.load('embedding_model_noavg_absolute.pth')

#Loading pre-saved weights to the model
model.load_state_dict(model_dict)


## Relative position model (vector model)

### Getting data

In [ ]:
# Getting vector data

def directional_vector(point1, point2):
    """Calculate the directional vector from point1 to point2."""
    return (point2[0] - point1[0], point2[1] - point1[1])

df = pd.read_csv('embedding_data.csv')

df_vector_list = []
bigx = []
bigy = []
bigp = []
bigseq = []

p0_x = []
p0_y = []
ps = np.unique(df["p"])

for seq in np.unique(df["Sequence"]):
    sub_seq_data = df[df["Sequence"] == seq]
    seq_points_x = []
    seq_points_y = []

    sub_p0 = sub_seq_data[sub_seq_data["p"] == 0 ]
    z0_p0_mean = sub_p0["Z0"].mean()
    z1_p0_mean = sub_p0["Z1"].mean()
    for p in ps:
        sub2 = sub_seq_data[sub_seq_data["p"] == p]
        z0_mean = sub2["Z0"].mean()
        z1_mean = sub2["Z1"].mean()
        seq_points_x.append(z0_mean)
        seq_points_y.append(z1_mean)
        bigp += [p]
        #Need to get the p=0 points here
        p0_x += [z0_p0_mean]
        p0_y += [z1_p0_mean]
        bigseq +=[seq]


    bigx+= seq_points_x
    bigy += seq_points_y
    coordinates = list(zip(seq_points_x, seq_points_y))
    p0_coord = (z0_p0_mean, z1_p0_mean)
    vectors = [directional_vector(coordinates[i], p0_coord) for i in range(len(coordinates))]
    df_vector_list += vectors
    
start_point_and_p = [] #Getting the start point for the vector and the p value for that point


for row in list(zip(p0_x, p0_y, bigp)):
    start_point_and_p += [row]
    
    

In [ ]:
# Train test val splitting for vector data
test_val_splitter = GroupShuffleSplit(n_splits = 1, test_size = 0.6, random_state = 0)

#DIFFERENT FROM ABSOLUTE MODEL
nia = np.array(start_point_and_p)
noa = np.array(df_vector_list)
seq_array = np.array(bigseq)

test_val_indices, train_indices = next(test_val_splitter.split(nia, noa, groups = seq_array))

x_test_val, x_train_rel = nia[test_val_indices], nia[train_indices]
y_test_val, y_train_rel = noa[test_val_indices], noa[train_indices]
groups_test_val = seq_array[test_val_indices]

val_test_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
val_indices, test_indices = next(val_test_splitter.split(x_test_val, y_test_val, groups=groups_test_val))

x_val_rel, x_test_rel = x_test_val[val_indices], x_test_val[test_indices]
y_val_rel, y_test_rel = y_test_val[val_indices], y_test_val[test_indices]

test_seqs = seq_array[test_indices]
val_seqs = seq_array[val_indices]
#Turning them into tensors
x_train_rel = torch.Tensor(x_train_rel)
x_val_rel = torch.Tensor(x_val_rel)
y_train_rel = torch.Tensor(y_train_rel)
y_val_rel = torch.Tensor(y_val_rel)
x_test_rel = torch.Tensor(x_test_rel)
y_test_rel = torch.Tensor(y_test_rel)

### Training model

In [ ]:
# Training model
with open('best_parameters_absolute.json', 'r') as f:
    best_parameters = json.load(f)

model_relative, optimizer = generate_model_and_optimizer(best_parameters)
x_train_rel = torch.Tensor(x_train_rel)
x_val_rel = torch.Tensor(x_val_rel)
y_train_rel = torch.Tensor(y_train_rel)
y_val_rel = torch.Tensor(y_val_rel)
x_test_rel = torch.Tensor(x_test_rel)
y_test_rel = torch.Tensor(y_test_rel)

data = (x_train_rel, x_val_rel, y_train_rel, y_val_rel)
train_model(model_relative, data, optimizer, best_parameters['n_epochs'])
# train rmse
y_pred = model_relative(x_train_rel).detach()
rmse_train = torch.sqrt( torch.mean( (y_pred - y_train_rel)**2 ) ).item()
# validation rmse
y_pred = model_relative(x_val_rel).detach()
rmse_val = torch.sqrt( torch.mean( (y_pred - y_val_rel)**2 ) ).item()
# test rmse
y_pred = model_relative(x_test_rel).detach()
rmse_test = torch.sqrt( torch.mean( (y_pred - y_test_rel)**2 ) ).item()
# report
print(f'\ntrain: {rmse_train:.3e} / val: {rmse_val:.3e} / test: {rmse_test:.3e}')

In [ ]:
# Saving model 
#Saving model and best parameters
torch.save(model.state_dict(), 'embedding_model_noavg_relative.pth')
with open('best_parameters_relative.json', 'w') as f:
    json.dump(best_parameters, f)

### Finding RMSE as a f(p)

In [ ]:
x_train_rel = torch.Tensor(x_train_rel)
x_val_rel = torch.Tensor(x_val_rel)
y_train_rel = torch.Tensor(y_train_rel)
y_val_rel = torch.Tensor(y_val_rel)
x_test_rel = torch.Tensor(x_test_rel)
y_test_rel = torch.Tensor(y_test_rel)

#Train
y_pred_train = model_relative(x_train_rel).detach()

#Test
y_pred_test = model_relative(x_test_rel).detach()

#Val
y_pred_val = model_relative(x_val_rel).detach()

train_rmse_list = []
test_rmse_list = []
val_rmse_list = []

for p in ps:
    _, rmse_train = get_rmse_by_p(p, y_pred_train, x_train_rel, y_train_rel)
    _, rmse_test = get_rmse_by_p(p, y_pred_test, x_test_rel, y_test_rel)
    _, rmse_val = get_rmse_by_p(p, y_pred_val, x_val_rel, y_val_rel)

    train_rmse_list += [rmse_train]
    test_rmse_list += [rmse_test]
    val_rmse_list +=[rmse_val]

    print(f'\n p: {p}  train: {rmse_train:.3e} / val: {rmse_val:.3e} / test: {rmse_test:.3e}')

plt.plot(ps, train_rmse_list, label = "Train")
plt.plot(ps, test_rmse_list, label = "Test")
plt.plot(ps, val_rmse_list, label = "Validation")
plt.legend()
plt.title("Relative Position Model RMSE")
plt.show()

#### To load the model without retraining

In [ ]:
# If I want to reload the model without training
# If I want to load the model without retraining:
with open('best_parameters_relative.json', 'r') as f:
    best_parameters = json.load(f)

#Initializing the model (maybe not the right wording here)
model_relative, optimizer = generate_model_and_optimizer(best_parameters)
model_dict = torch.load('embedding_model_noavg_relative.pth')

#Loading pre-saved weights to the model
model_relative.load_state_dict(model_dict)

# Next stuff

## Finding gradients

### Absolute model

In [ ]:
# Loading absolute model
with open('best_parameters_absolute.json', 'r') as f:
    best_parameters = json.load(f)

#Initializing the model (maybe not the right wording here)
model_absolute, optimizer = generate_model_and_optimizer(best_parameters)
model_dict = torch.load('embedding_model_noavg_absolute.pth')

#Loading pre-saved weights to the model
model_absolute.load_state_dict(model_dict)

In [ ]:
#Get grid points

from alphashape import alphashape

ps = np.unique(df["p"])
z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)
max_x = max(z_array[:,0])
max_y = max(z_array[:,1])

min_x = min(z_array[:,0])
min_y = min(z_array[:,1])

x_range = np.linspace(min_x, max_x, 30)
y_range = np.linspace(min_y, max_y, 30)
X,Y = np.meshgrid(x_range, y_range)

mesh_point_set = []

#Making an array of all the points
for i, row in enumerate(X): 
    row_mesh = np.concatenate((row.reshape(-1,1), Y[i].reshape(-1,1)), axis = 1)
    mesh_point_set.append(row_mesh)

mesh_set = np.vstack(mesh_point_set)


alpha_shape = alphashape(z_array, alpha = 1)
#getting mesh points that are actually in my alpha shape
alpha_shape_mesh_points = []
for point in mesh_set:
    point_shape = Point(point)
    if alpha_shape.contains(point_shape) == True:
        alpha_shape_mesh_points.append(point)
alpha_shape_mesh = np.vstack(alpha_shape_mesh_points)


In [ ]:
#Finding norms and plotting

zero_arr = np.full((alpha_shape_mesh.shape[0], 1), 0)
grid_pts = np.hstack((alpha_shape_mesh, zero_arr))

row_and_norm = []

for row in grid_pts:
    point = torch.Tensor(row)
    point.requires_grad_(True)
    output = model_absolute(point)
    scalar_output = torch.sum(output)
    scalar_output.backward()
    dx = point.grad[0].item()
    dy = point.grad[1].item()
    gradient_norm = torch.norm(point.grad)

    x = point[0].detach().numpy()
    y = point[1].detach().numpy()
    norm = gradient_norm.detach().numpy()

    row_and_norm += [np.array((x, y, norm))]

row_and_norm_arr = np.vstack(row_and_norm)
plt.scatter(row_and_norm_arr[:,0], row_and_norm_arr[:,1], c = row_and_norm_arr[:,2], cmap = 'viridis')
plt.colorbar()
plt.title("Norm of gradients using absolute model")
plt.xlabel("Z0")
plt.ylabel("Z1")
plt.show()


### Relative model

In [ ]:
# If I want to reload the model without training
# If I want to load the model without retraining:
with open('best_parameters_relative.json', 'r') as f:
    best_parameters = json.load(f)

#Initializing the model (maybe not the right wording here)
model_relative, optimizer = generate_model_and_optimizer(best_parameters)
model_dict = torch.load('embedding_model_noavg_relative.pth')

#Loading pre-saved weights to the model
model_relative.load_state_dict(model_dict)

In [ ]:
#Get grid points

from alphashape import alphashape

ps = np.unique(df["p"])
z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)
max_x = max(z_array[:,0])
max_y = max(z_array[:,1])

min_x = min(z_array[:,0])
min_y = min(z_array[:,1])

x_range = np.linspace(min_x, max_x, 30)
y_range = np.linspace(min_y, max_y, 30)
X,Y = np.meshgrid(x_range, y_range)

mesh_point_set = []

#Making an array of all the points
for i, row in enumerate(X): 
    row_mesh = np.concatenate((row.reshape(-1,1), Y[i].reshape(-1,1)), axis = 1)
    mesh_point_set.append(row_mesh)

mesh_set = np.vstack(mesh_point_set)


alpha_shape = alphashape(z_array, alpha = 1)
#getting mesh points that are actually in my alpha shape
alpha_shape_mesh_points = []
for point in mesh_set:
    point_shape = Point(point)
    if alpha_shape.contains(point_shape) == True:
        alpha_shape_mesh_points.append(point)
alpha_shape_mesh = np.vstack(alpha_shape_mesh_points)


In [ ]:
zero_arr = np.full((alpha_shape_mesh.shape[0], 1), 0)
grid_pts = np.hstack((alpha_shape_mesh, zero_arr))

row_and_norm = []

for row in grid_pts:
    point = torch.Tensor(row)
    point.requires_grad_(True)
    output = model_relative(point)
    scalar_output = torch.sum(output)
    scalar_output.backward()
    dx = point.grad[0].item()
    dy = point.grad[1].item()
    gradient_norm = torch.norm(point.grad)

    x = point[0].detach().numpy()
    y = point[1].detach().numpy()
    norm = gradient_norm.detach().numpy()

    row_and_norm += [np.array((x, y, norm))]

row_and_norm_arr = np.vstack(row_and_norm)
plt.scatter(row_and_norm_arr[:,0], row_and_norm_arr[:,1], c = row_and_norm_arr[:,2], cmap = 'viridis')
plt.colorbar()
plt.title("Norm of gradients using relative/vector model")
plt.xlabel("Z0")
plt.ylabel("Z1")
plt.show()

# KDE

In [ ]:
#Uniform sampling in p space and xy space, they'll be different. 
from scipy.stats import gaussian_kde

lines = []

for seq in np.unique(df["Sequence"]):
    sub_seq_data = df[df["Sequence"] == seq]
    for run in np.unique(df["Run number"]):
        sub_run_data = sub_seq_data[sub_seq_data["Run number"] == run]

        x = sub_run_data["Z0"]
        y = sub_run_data["Z1"]
        cols = sub_run_data["p"]

        line = np.vstack((x,y))
        lines += [line]
    

In [ ]:
# Getting points evently along a streamline, evenly along x and y 

from shapely.geometry import LineString, Point

all_lines = []

for seq in np.unique(df["Sequence"]):
    sub_seq_data = df[df["Sequence"] == seq]
    for run in np.unique(df["Run number"]):
        sub_run_data = sub_seq_data[sub_seq_data["Run number"] == run]

        x = sub_run_data["Z0"]
        y = sub_run_data["Z1"]
        cols = sub_run_data["p"]

        points = np.array([x, y]).T.reshape(-1, 1, 2)

            
        segments = np.concatenate([points[:-1], points[1:]], axis=1)

        all_lines +=[segments]
            

sampled_lines_all = []
for idk in all_lines:

# Example: Replace this with your actual line segments
    line_segments = np.array(idk)
    # Convert line segments to LineString objects
    # Interpolate points along each line segment
    sampled_points = []
    for segment in line_segments:
        line = LineString(segment)
        total_length = line.length
        desired_interval = 0.5  # Adjust as needed

        current_length = 0
        while current_length < total_length:
            point = line.interpolate(current_length)
            x, y = point.xy[0][0], point.xy[1][0]
            sampled_points.append((x, y))
            current_length += desired_interval

    # Convert sampled points to a NumPy array
    sampled_points_array = np.array(sampled_points)
    sampled_lines_all +=[sampled_points_array]

# Plot the original line segments
fig, ax = plt.subplots(figsize=(10,7))
for segment in line_segments:
    ax.plot(segment[:, 0], segment[:, 1], color='blue', marker='o')

# Plot the sampled points
ax.scatter(sampled_points_array[:, 0], sampled_points_array[:, 1], color='red', marker='x')
z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)
alpha_shape = alphashape(z_array, alpha = 1)
plt.plot(*alpha_shape.exterior.xy, color='black', linewidth=2,alpha = 0.7, label='Alpha Shape')
plt.title("Interval = " + str(desired_interval))
plt.show()

even_xy_mesh = np.vstack(sampled_lines_all)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


#Get grid points

from alphashape import alphashape

ps = np.unique(df["p"])
z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)
max_x = max(z_array[:,0])
max_y = max(z_array[:,1])

min_x = min(z_array[:,0])
min_y = min(z_array[:,1])

x_range = np.linspace(min_x, max_x, 50)
y_range = np.linspace(min_y, max_y, 50)
X,Y = np.meshgrid(x_range, y_range)

mesh_point_set = []

#Making an array of all the points
for i, row in enumerate(X): 
    row_mesh = np.concatenate((row.reshape(-1,1), Y[i].reshape(-1,1)), axis = 1)
    mesh_point_set.append(row_mesh)

mesh_set = np.vstack(mesh_point_set)


alpha_shape = alphashape(z_array, alpha = 1)
#getting mesh points that are actually in my alpha shape
alpha_shape_mesh_points = []
for point in mesh_set:
    point_shape = Point(point)
    if alpha_shape.contains(point_shape) == True:
        alpha_shape_mesh_points.append(point)
alpha_shape_mesh = np.vstack(alpha_shape_mesh_points)


empty_list = []

for line in lines:
    idk= np.vstack((line[0], line[1])).T
    empty_list += [idk]

#lines = np.asarray(lines)

# Flatten the arrays and concatenate
#all_points = np.concatenate([line.T for line in lines])

all_points = empty_list

your_mesh = even_xy_mesh

# Perform kernel density estimation
kde = gaussian_kde(your_mesh.T)

#grid_coordinates = alpha_shape_mesh

density_values = kde(your_mesh.T)

# Reshape the density values to match the shape of your_mesh


# Plot the original lines

#for line in lines:
        #plt.plot(line[0, :], line[1, :], color='gray', alpha=0.2, zorder = 0)

line = lines[0]
plt.plot(line[0, :], line[1, :], color='gray', alpha=0.2, zorder = 0)

# Plot the KDE density
z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)
alpha_shape = alphashape(z_array, alpha = 1)
plt.plot(*alpha_shape.exterior.xy, color='black', linewidth=2,alpha = 0.7, label='Alpha Shape')

plt.scatter(your_mesh[:, 0], your_mesh[:, 1], c=density_values, cmap='viridis', marker='.', s=10, zorder = 1)
plt.colorbar(label='Density')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Kernel Density Estimation for Lines')
plt.show()

### KDE for even spacing along p

In [ ]:
one_run_df = df[df["Run number"] == 1]
downselected_seq = np.random.choice(np.unique(one_run_df["Sequence"]), size = 70, replace = False)

xs = []
ys = []

for thing in downselected_seq:
    sampleset = one_run_df[one_run_df["Sequence"] == thing]
    x = list(sampleset["Z0"])
    y = list(sampleset["Z1"])

    xs += [x]
    ys += [y]

x_arr = np.hstack(xs)
y_arr = np.hstack(ys)

streamline_grid = np.vstack((x_arr, y_arr)).T

In [ ]:
#Get grid points

from alphashape import alphashape

ps = np.unique(df["p"])
z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)

alpha_shape_mesh = streamline_grid

empty_list = []

for line in lines:
    idk= np.vstack((line[0], line[1])).T
    empty_list += [idk]

#lines = np.asarray(lines)

# Flatten the arrays and concatenate
#all_points = np.concatenate([line.T for line in lines])

all_points = empty_list

your_mesh = alpha_shape_mesh

# Perform kernel density estimation
kde = gaussian_kde(your_mesh.T)

#grid_coordinates = alpha_shape_mesh

density_values = kde(your_mesh.T)

# Reshape the density values to match the shape of your_mesh


# Plot the original lines

for line in lines:
        plt.plot(line[0, :], line[1, :], color='gray', alpha=0.05)

# Plot the KDE density
z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)
alpha_shape = alphashape(z_array, alpha = 1)
plt.plot(*alpha_shape.exterior.xy, color='black', linewidth=2,alpha = 0.7, label='Alpha Shape')

plt.scatter(your_mesh[:, 0], your_mesh[:, 1], c=density_values, cmap='viridis', marker='.', s=10)
plt.colorbar(label='Density')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Kernel Density Estimation for Lines')
plt.show()

# Hex grid

### Gets the grid

In [ ]:
#First go at making hex grid points and finding neighbors

def create_hex_grid(radius, x_min, y_min, x_max, y_max):
    """
    Create a hex grid of points within a specified bounding box.

    Parameters:
    - radius: Radius of the hexagon.
    - x_min: Minimum x-coordinate of the bounding box.
    - y_min: Minimum y-coordinate of the bounding box.
    - x_max: Maximum x-coordinate of the bounding box.
    - y_max: Maximum y-coordinate of the bounding box.

    Returns:
    - List of hex grid points.
    """
    hex_points = []
    hex_width = radius * np.sqrt(3)
    hex_height = 2 * radius

    # Calculate the number of rows and columns based on the bounding box
    num_rows = int((y_max - y_min) / (1.5 * radius))
    num_cols = int((x_max - x_min) / hex_width)

    for row in range(num_rows):
        for col in range(num_cols):
            x = col * hex_width + x_min
            y = row * 1.5 * radius + y_min

            # Offset every other row
            if col % 2 == 1:
                y += 0.75 * radius

            hex_points.append((x, y))

    return hex_points

#Parameters for hex grid and getting hex grid on a rectangle
radius = 1.0
x_min, y_min, x_max, y_max = -7.0, -7.0, 25.0, 15.0
hex_points = create_hex_grid(radius, x_min, y_min, x_max, y_max)

#Getting mesh points that are actually in my alpha shape
alpha_shape_mesh_points = []
for point in hex_points:
    point_shape = Point(point)
    if alpha_shape.contains(point_shape) == True:
        alpha_shape_mesh_points.append(point)
alpha_shape_mesh_hex = np.vstack(alpha_shape_mesh_points)


#Plotting hex grid
z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)
alpha_shape = alphashape(z_array, alpha = 1)
plt.plot(*alpha_shape.exterior.xy, color='black', linewidth=2,alpha = 0.7, label='Alpha Shape')
plt.scatter(alpha_shape_mesh_hex[:,0], alpha_shape_mesh_hex[:,1])
plt.show()


In [ ]:
# Gets the closest hex grid points to a certain point inside the alphashape

def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b)**2))

def get_neighbors(center, all_points, radius, alphashape):
    neighbors = []

    for point in all_points:
        if not np.array_equal(point, center) and euclidean_distance(center, point) <= radius:
            neighbors.append(point)

    neighbor_mesh = []
    for point in neighbors:
        point_shape = Point(point)
        if alphashape.contains(point_shape) == True:
            neighbor_mesh.append(point)
    neighbor_mesh = np.vstack(neighbor_mesh)

    return neighbor_mesh




# Choose a center point for which you want to find neighbors
center_point = alpha_shape_mesh_hex[25]
radius = 2

z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)
alpha_shape = alphashape(z_array, alpha = 1)
# Get neighbors of the center point within the specified radius
neighbors = get_neighbors(center_point, hex_points, radius, alpha_shape)

print(f"Neighbors of {center_point} in alphashape: {neighbors}")




plt.plot(*alpha_shape.exterior.xy, color='black', linewidth=2,alpha = 0.7, label='Alpha Shape')
plt.scatter(center_point[0], center_point[1])
plt.scatter(neighbors[:,0], neighbors[:,1])
plt.show()

In [ ]:
# Assuming you already have hex grid

# Loops through every point in mesh

def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b)**2))

def get_neighbors(center, all_points, radius, alphashape):
    neighbors = []

    for point in all_points:
        if not np.array_equal(point, center) and euclidean_distance(center, point) <= radius:
            neighbors.append(point)

    neighbor_mesh = []
    for point in neighbors:
        point_shape = Point(point)
        if alphashape.contains(point_shape) == True:
            neighbor_mesh.append(point)
    neighbor_mesh = np.vstack(neighbor_mesh)

    return neighbor_mesh

z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)
alpha_shape = alphashape(z_array, alpha = 1)
radius = 2

for center_point in alpha_shape_mesh_hex:
    neighbors = get_neighbors(center_point, hex_points, radius, alpha_shape)
    print(f"Neighbors of {center_point}: {neighbors}")